# Algotytm przecinania się odcinków na płaszczyźnie

In [109]:
import numpy as np
from bitalg.visualizer.main import Visualizer
from collections import defaultdict
from sortedcontainers import SortedSet

# Przykładowa wizualizacja punktów i ich przecięć

In [2]:
def draw_example_1():
    vis = Visualizer()
    line_segments = ((-0.5, 0.5), (8.5, 3.5),
                     (1, 3), (7, 5),(2, 4), (5, 1),
                     (4.5, 3), (6.5, 6),
                     (0, 5), (5.5, 5.5))

    vis.add_line_segment(line_segments)
    vis.show()
def draw_example_2():
    vis = Visualizer()
    line_segments = ((-0.5, 0.5), (8.5, 3.5),
                     (1, 3), (7, 5),(2, 4), (5, 1),
                     (4.5, 3), (6.5, 6),
                     (0, 5), (5.5, 5.5))
    points = [(4, 2), (2.5, 3.5),(5.5, 4.5)]
    vis.add_line_segment(line_segments)
    vis.add_point(points, color='red')
    vis.show()

In [ ]:
draw_example_1()

In [ ]:
draw_example_2()

# Generowanie losowych odcinków na płaszczyźnie

In [110]:
def generate_uniform_sections(max_x, max_y, n): # mozna poprawic
    """
    Funkcja generuje odcinki o współrzędnych rzeczywistych w postaci par punktów. 
    Żaden wygenerowany odcinek nie jest odcinkiem pionowym.
    Żadne dwa odcinki nie mają swoich końców o takiej samej współrzędnej x.
    Zakres współrzędnych: x -> (0, max_x), y -> (0, max_y)
    :param max_x: określa maksymalną wartość współrzednej x jaka może zostać wylosowana
    :param max_y: określa maksymalną wartość współrzednej y jaka może zostać wylosowana
    :param n: ilość generowanych odcinków
    :return: tablica odcinków w postaci krotek zawierających parę krotek współrzędnych punktów końcowych odcinków
    np. [((x1, y1), (x2, y2)), ((x3, y3), (x4, y4)),...]
    """
    Points = []
    used_x = defaultdict()
    for _ in range(n):
        x_1 = np.random.uniform(0,max_x)
        y_1 = np.random.uniform(0,max_y)       
        x_2 = np.random.uniform(0,max_x)
        y_2 = np.random.uniform(0,max_y)
        while x_2 == x_1 and used_x(x_2) == 0 and used_x(x_1) == 0:
            x_2 = np.random.uniform(0,max_x)
            used_x[x_2]=1
        Points.append([[x_1,y_1],[x_2,y_2]])
    return Points

Losowanie losowych 20 odcników i wizualizacja

In [ ]:
Twenty = generate_uniform_sections(1000, 1000, 20)
def draw_exercise(Points):
    vis = Visualizer()
    vis.add_line_segment(Points)
    vis.show()
draw_exercise(Twenty)
#print(Twenty)

Funckja do interaktywnego dodawania odcinków klikając myszką lub wpisując współrzędne

In [142]:
import matplotlib.pyplot as plt
from matplotlib.widgets import Button, TextBox
import json
import os
import tkinter as tk
from tkinter import filedialog
%matplotlib tk

all_sets_of_segments = []
current_set_of_segments = []
current_segment = []
input_mode = False  
limits_mode = False 

initial_xlim = [-100, 100]
initial_ylim = [-100, 100]
current_xlim = initial_xlim[:]
current_ylim = initial_ylim[:]

def draw_interactive_segments():
    global current_xlim, current_ylim, current_segment, current_set_of_segments

    ax.cla() 

    if len(current_segment) == 2:
        current_set_of_segments.append(current_segment[:])
        current_segment = []

    if current_segment:
        x, y = zip(*current_segment)
        ax.plot(x, y, 'ro-', markersize=5, label='Current Segment')

    for segment in current_set_of_segments:
        if len(segment) == 2:
            x, y = zip(*segment)
            ax.plot(x, y, 'bo-', markersize=5, label='Completed Segment')

    ax.set_title(" ")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")

    ax.set_xlim(current_xlim)
    ax.set_ylim(current_ylim)

    plt.draw()

def on_click(event):
    if event.button == 1 and event.inaxes and not input_mode and not limits_mode:
        current_segment.append((event.xdata, event.ydata))
        draw_interactive_segments()

def add_point_from_input(event=None):
    global current_xlim, current_ylim
    if input_mode and not limits_mode:
        try:
            x = float(text_box_x.text)
            y = float(text_box_y.text)
            current_segment.append((x, y))
            
            x_margin = 0.1 * (current_xlim[1] - current_xlim[0]) 
            y_margin = 0.1 * (current_ylim[1] - current_ylim[0])
            
            if x < current_xlim[0]:
                current_xlim[0] = x - x_margin
            if x > current_xlim[1]:
                current_xlim[1] = x + x_margin
            if y < current_ylim[0]:
                current_ylim[0] = y - y_margin
            if y > current_ylim[1]:
                current_ylim[1] = y + y_margin
            
            draw_interactive_segments()
        except ValueError:
            print("Invalid coordinates.")

def on_key(event):
    global input_mode, limits_mode, current_segment, current_set_of_segments
    if event.key == 's':
        save_polygon()
    elif event.key == 'b':
        if (current_set_of_segments or current_segment) and not input_mode and not limits_mode:
            if current_segment:
                current_segment.clear()
            else:
                current_set_of_segments.pop()
            draw_interactive_segments()
    elif event.key == 'i': 
        toggle_input_mode()
    elif event.key == 'c': 
        clear_polygon()

def toggle_input_mode(event=None):
    global input_mode
    input_mode = not input_mode
    update_input_mode_display()

def update_input_mode_display():
    if input_mode:
        input_mode_display.label.set_text("Input Mode: ON")
    else:
        input_mode_display.label.set_text("Input Mode: OFF")

    plt.draw()

def save_polygon():
    if not current_set_of_segments:
        print("No points to save.")
        return
    if current_segment:
        print("Improper point left.")
        return

    root = tk.Tk()
    root.withdraw()

    json_filepath = filedialog.asksaveasfilename(
        defaultextension=".json",
        filetypes=[("JSON files", ".json"), ("All files", ".*")],
        initialdir=".",
        title="Save Polygon As JSON"
    )

    if json_filepath:
        with open(json_filepath, "w") as file:
            all_sets_of_segments.append(current_set_of_segments[:])
            json.dump(current_set_of_segments, file)
        print(f"Saved polygon to file {json_filepath}")
    else:
        print("Save operation was cancelled.")

def clear_polygon(event=None):
    global current_set_of_segments, current_xlim, current_ylim
    if not input_mode and not limits_mode:
        current_set_of_segments.clear() 
        current_segment.clear()
        
     
        current_xlim = initial_xlim[:]
        current_ylim = initial_ylim[:]
        
        draw_interactive_segments()


fig, ax = plt.subplots()
ax.set_aspect('equal')
ax.set_xlim(initial_xlim)
ax.set_ylim(initial_ylim)
plt.draw()


fig.canvas.mpl_connect('button_press_event', on_click)

fig.canvas.mpl_connect('key_press_event', on_key)

ax_edit_mode = fig.add_axes([0.04, 0.75, 0.2, 0.05])
input_mode_display = Button(ax_edit_mode, 'Input Mode: OFF')
input_mode_display.on_clicked(toggle_input_mode)

ax_text_x = fig.add_axes([0.04, 0.70, 0.2, 0.05])
text_box_x = TextBox(ax_text_x, 'X:', initial="0")

ax_text_y = fig.add_axes([0.04, 0.65, 0.2, 0.05])
text_box_y = TextBox(ax_text_y, 'Y:', initial="0")

ax_add_point = fig.add_axes([0.04, 0.60, 0.2, 0.05])
btn_add_point = Button(ax_add_point, 'Add Point')
btn_add_point.on_clicked(add_point_from_input)

fig.canvas.mpl_connect('button_press_event', on_click)
fig.canvas.mpl_connect('key_press_event', on_key)

draw_interactive_segments()
plt.show(block=True)

### Wprowadzenie
Celem ćwiczenia jest implementacja i zapoznanie się z algorytmem wyznaczającym wszystkie przecięcia odcinków na płaszczyźnie

In [111]:
import matplotlib.pyplot as plt
import matplotlib.collections
from matplotlib.widgets import Button
from queue import PriorityQueue

Wprowadzamy pomocniczne klasy, w celu uzyskania wizualizacji działania algorytmu

In [112]:
#punkt
class Point: 
    def __init__(self, x, y, i1, i2):
        self.x = x
        self.y = y
        self.i1 = i1 # odcinke mna któym lezy punkt

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    def __gt__(self, other):
        return self.x < other.x
     
    def __hash__(self):
        return hash((self.x, self.y))


In [132]:
# odcinek
class Line:
    def __init__(self, start, end):
        self.start = start # początek odcinka
        self.end = end # koniec odcinka
        # prosta ax+b = y
        self.a = (self.start.y - self.end.y) / (self.start.x - self.end.x) 
        self.b = self.start.y - self.a * self.start.x 
        
    def __gt__(self, other):
        return self.x * self.a + self.b < other.x * other.a + other.b
    # tu tez
    def __hash__(self):
        return hash((self.start, self.end))
    
    def set_x(x): # obecna pozycja miotły
        Line.x = x
    

In [114]:
# przetrzymywanie punktów i lini
class Scene:
    def __init__(self, points=[], lines=[]):
        self.points=points
        self.lines=lines

In [115]:
# punkty w krotkach (x,y), takie same
class Points_Coll:
    def __init__(self, points, **kwargs): # słownik argumentów funkcji
        self.points = points
        self.kwargs = kwargs
    
    def add_points(self, points):
        self.points += points

In [116]:
# odcinki w krotkach ((x1,y1),x2,y2), takie same
class Lines_Coll:
    def __init__(self, lines, **kwargs):
        self.lines = lines
        self.kwargs = kwargs
        
    def add(self, line):
        self.lines.append(line)
        
    def get_collection(self):
        return matplotlib.collections.LineCollection(self.lines, **self.kwargs)

In [117]:
# obsluga przyciskow w narzedziu do rysowania z klikaniem(gify z klikaniem), funckja wygenerowana przez chatbota
def dist(point1, point2):
    return np.sqrt(np.power(point1[0] - point2[0], 2) + np.power(point1[1] - point2[1], 2))
class _Button_callback(object):
    def __init__(self, scenes):
        self.i = 0
        self.scenes = scenes
        self.adding_points = False
        self.added_points = []
        self.adding_lines = False
        self.added_lines = []
        self.adding_rects = False
        self.added_rects = []

    def set_axes(self, ax):
        self.ax = ax
        
    def next(self, _):
        self.i = (self.i + 1) % len(self.scenes)
        self.draw(autoscaling = True)

    def prev(self, _):
        self.i = (self.i - 1) % len(self.scenes)
        self.draw(autoscaling = True)
        
    def add_point(self, _):
        self.adding_points = not self.adding_points
        self.new_line_point = None
        if self.adding_points:
            self.adding_lines = False
            self.adding_rects = False
            self.added_points.append(Points_Coll([]))
                
    def add_line(self, _):   
        self.adding_lines = not self.adding_lines
        self.new_line_point = None
        if self.adding_lines:
            self.adding_points = False
            self.adding_rects = False
            self.added_lines.append(Lines_Coll([]))

    def add_rect(self,_):
        self.adding_rects = not self.adding_rects
        self.new_line_point = None
        if self.adding_rects:
            self.adding_points = False
            self.adding_lines = False
            self.new_rect()
    
    def new_rect(self):
        self.added_rects.append(Lines_Coll([]))
        self.rect_points = []
    def on_click(self, event):
        if event.inaxes != self.ax:
            return
        new_point = (event.xdata, event.ydata)
        if self.adding_points:
            self.added_points[-1].add_points([new_point])
            self.draw(autoscaling = False)
        elif self.adding_lines:
            if self.new_line_point is not None:
                self.added_lines[-1].add([self.new_line_point, new_point])
                self.new_line_point = None
                self.draw(autoscaling = False)
            else:
                self.new_line_point = new_point
        elif self.adding_rects:
            if len(self.rect_points) == 0:
                self.rect_points.append(new_point)
            elif len(self.rect_points) == 1:
                self.added_rects[-1].add([self.rect_points[-1], new_point])
                self.rect_points.append(new_point)
                self.draw(autoscaling = False)
            elif len(self.rect_points) > 1:
                if dist(self.rect_points[0], new_point) < (np.mean([self.ax.get_xlim(), self.ax.get_ylim()])*0,15):
                    self.added_rects[-1].add([self.rect_points[-1], self.rect_points[0]])
                    self.new_rect()
                else:    
                    self.added_rects[-1].add([self.rect_points[-1], new_point])
                    self.rect_points.append(new_point)
                self.draw(autoscaling = False)
    def draw(self, autoscaling = True):
        xlim = self.ax.get_xlim()
        ylim = self.ax.get_ylim()
        self.ax.clear()
        for collection in (self.scenes[self.i].points + self.added_points):
            if len(collection.points) > 0:
                self.ax.scatter(*zip(*(np.array(collection.points))), **collection.kwargs)
        for collection in (self.scenes[self.i].lines + self.added_lines + self.added_rects):
            self.ax.add_collection(collection.get_collection())
        self.ax.autoscale(autoscaling)
        if not autoscaling:
            self.ax.set_xlim(xlim)
            self.ax.set_ylim(ylim)
        plt.draw()

In [118]:
def det(a, b):return a[0] * b[1] - a[1] * b[0]

In [119]:
def intersect(l1, l2): # sprawdza, czy odcinki się tna
    (x1, y1) = (l1.start.x, l1.start.y)
    (x2, y2) = (l1.end.x, l1.end.y)
    (x3, y3) = (l2.start.x, l2.start.y)
    (x4, y4) = (l2.end.x, l2.end.y)
    d = det([(x1-x2), (x3-x4)], [(y1-y2), (y3-y4)])
    t1 = det([(x1-x3), (x3-x4)], [(y1-y3), (y3-y4)]) / d
    t2 = -det([(x1-x2), (x1-x3)], [(y1-y2), (y1-y3)]) / d
    
    if 0 <= t1 <= 1 and 0 <= t2 <= 1:
        return Point(l1.start.x + t1 * (l1.end.x - l1.start.x), l1.start.y + t1 * (l1.end.y - l1.start.y), l1.start.i1, l2.start.i1)
    return None

In [120]:
def save(plot, name): ## zapisywanie do pliku
    segments = []
    for i in range(len(plot.get_added_lines())):
        for line in plot.get_added_lines()[i].lines:
            segments.append(line)

    with open(f'{name}.json', 'w') as file:
       file.write(json.dumps(segments))
        
def load(name):  # odczytywanie z pliku json i przetwarzanie na listę obiektów Line
    segments = []
    for segment in name:
        id = len(segments)
        start = Point(segment[0][0], segment[0][1], id, id)
        end = Point(segment[1][0], segment[1][1], id, id)
        if start.x == end.x: continue
        elif start.x > end.x: 
            start, end = end, start
        segments.append(Line(start, end))
    return segments

In [121]:
def add_scene(p, result, data, broom, scenes, T,num): # obecny stan, klatka gifa
        if num ==1: # do pojedynczych
            scenes.append(Scene([Points_Coll([result.copy()]),
            Points_Coll([(p.x, p.y)], color='purple')],
            [Lines_Coll(data, color='black'),
            Lines_Coll([broom], color='red'),
            Lines_Coll([[(l.start.x, l.start.y), (l.end.x, l.end.y)] for l in T], color='green')]))
        elif num == 2: # do wszysktich
            scenes.append(Scene([Points_Coll([(a[0], a[1]) for a in result]),
            Points_Coll([(p.x, p.y)], color='purple')],
            [Lines_Coll(data, color='black'),
            Lines_Coll([broom], color='red'),
            Lines_Coll([[(l.start.x, l.start.y), (l.end.x, l.end.y)] for l in T], color='green')]))

def classify_to_pq(segment, q, max_size):# liczenie 
    bigger, smaller = None, None
    
    if segment.start.x > segment.end.x: 
        bigger, smaller = segment.start, segment.end
    else: 
        bigger, smaller = segment.end, segment.start
    
    q.put((bigger.x, 1, segment, bigger, None))
    q.put((smaller.x, 0, segment, smaller, None))
    max_size = max( max_size, abs(segment.start.y), abs(segment.end.y))
    return max_size

#### Funckja obliczająca pierwsze przecięcie wraz z wizualizacja

In [122]:
def is_intersect(data): 
    segments = load(data) # ładowanie z JSONA
    Q = PriorityQueue() # zdarzenia
    T = SortedSet() # 'aktywne' odcinki
    result = [None,None]
    scenes= [] # klatki gifow
    size = 0
    
    for segment in segments:
        size = classify_to_pq(segment, Q, size) # pozycjonowanie

    Line.set_x(0) # miotła w pozycji defaultowej
    # Q - x - współrzędna zdarzenia, side - 0 początkek, 1- kkoniec
    # segment - odcniek, point - obkiet Point reprezentujący odcniek
    # 5 pole dla następnej funckji
    while not Q.empty(): # dopóki koniec lub przecięcie
        x1, side, _, p, _ = Q.get()
        broom = [(p.x, -size), (p.x, size)] # obecne polozenie miotly
        line = segments[p.i1]
        add_scene(p,result, data, broom, scenes, T,1)

        if side == 0: # początek odcinka
            Line.set_x(x1)
            T.add(line)
            added = T.index(line)
            #add_scene(p,result, data, broom, scenes, T,1)
            #jesli przecina sie z sasiadenim wyzej, nizej
            if added > 0: 
                cut = intersect(line, T[added-1])
                if cut is not None:
                    result = [cut.x, cut.y]
                    scenes = [Scene([Points_Coll([result.copy()])],
                        [Lines_Coll(data)])] + scenes
                    break
            
            elif added < len(T) - 1:
                cut = intersect(line, T[added+1])
                if cut is not None:
                    result = [cut.x, cut.y]
                    scenes = [Scene([Points_Coll([result.copy()])],
                        [Lines_Coll(data)])] + scenes
                    break
            add_scene(p,result, data, broom, scenes, T,1)
        elif side == 1: 
            idx = T.index(line)
            if idx > 0 and idx < len(T) - 1:
                cut = intersect(T[idx-1], T[idx+1])
                if  cut is not None: 
                    result = [cut.x, cut.y]
                    scenes = [Scene([Points_Coll([result.copy()])],
                        [Lines_Coll(data)])] + scenes
                    break
            T.remove(line)
            
            add_scene(p,result, data, broom, scenes, T,1)
    add_scene(p,result, data, broom, scenes, T,1)
    scenes = [Scene([Points_Coll([result.copy()])],
                    [Lines_Coll(data)])] + scenes
    scenes = scenes[1:]
    
    # return result
    return result[0] != None, scenes, result

Wizualizacja

In [123]:
# Klasa do wizualizacji
import json
class Plot:
    def __init__(self, scenes = [Scene()], points = [], lines = [], json = None):
        if json is None:
            self.scenes = scenes
            if points or lines:
                self.scenes[0].points = points
                self.scenes[0].lines = lines
        else:
            self.scenes = [Scene([Points_Coll(pointsCol) for pointsCol in scene["points"]], 
                                 [Lines_Coll(linesCol) for linesCol in scene["lines"]]) 
                           for scene in json.loads(json)]
    def __configure_buttons(self):
        plt.subplots_adjust(bottom=0.3)
        ax_prev = plt.axes([0.1, 0.05, 0.15, 0.08])
        ax_next = plt.axes([0.3, 0.05, 0.15, 0.08])
        b_next = Button(ax_next, 'Next')
        b_next.on_clicked(self.callback.next)
        b_prev = Button(ax_prev, 'Prev')
        b_prev.on_clicked(self.callback.prev)
        return [b_prev, b_next]
    
    def add_scene(self, scene):
        self.scenes.append(scene)
    
    def add_scenes(self, scenes):
        self.scenes = self.scenes + scenes

    def toJson(self):
        return json.dumps([{"points": [np.array(pointCol.points).tolist() for pointCol in scene.points], 
                          "lines":[linesCol.lines for linesCol in scene.lines]} 
                         for scene in self.scenes])    
    
    # Rysowanie gifów
    def draw(self):
        plt.close()
        fig = plt.figure()
        self.callback = _Button_callback(self.scenes)
        self.widgets = self.__configure_buttons()
        ax = plt.axes(autoscale_on = False)
        self.callback.set_axes(ax)
        fig.canvas.mpl_connect('button_press_event', self.callback.on_click)
        plt.show()
        self.callback.draw()

In [124]:
# program do robienia automatycznych gifow
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

def create_sweep_gif(plot, output_filename='cos.gif', duration=500, loop=0):
    fig, ax = plt.subplots(figsize=(12, 9))
    
    frames = []
    
    first_scene_points = np.array([point for scene in plot.scenes for point_collection in scene.points for point in point_collection.points])
    x_min, x_max = np.min(first_scene_points[:, 0]), np.max(first_scene_points[:, 0])
    y_min, y_max = np.min(first_scene_points[:, 1]), np.max(first_scene_points[:, 1])
    
    padding = 0.1
    x_margin = (x_max - x_min) * padding
    y_margin = (y_max - y_min) * padding
    
    ax.set_xlim(x_min - x_margin, x_max + x_margin)
    ax.set_ylim(y_min - y_margin, y_max + y_margin)
   
    for scene_index, scene in enumerate(plot.scenes):
        
        ax.clear()
        
       
        for point_collection in scene.points:
           
            points = np.array(point_collection.points)
            
           
            if len(points) > 0:
                
                if scene_index == 0:
                    ax.scatter(points[:, 0], points[:, 1], color='blue', s=50,)
                else:
                    
                    ax.scatter(points[:, 0], points[:, 1], color='pink', s=250, marker='.',zorder = 3)
        
        
        for line_index, line_collection in enumerate(scene.lines):
           
            lines = line_collection.lines
            
            for line in lines:
                line = np.array(line)
                
                
                if scene_index == 0:
                    ax.plot(line[:, 0], line[:, 1], color='blue', linewidth=2)
                else:
                   
                    if line_index == 0:
                    
                        ax.plot(line[:, 0], line[:, 1], color='red', linewidth=3)
                    elif line_index == 1:
                      
                        ax.plot(line[:, 0], line[:, 1], color='black', linewidth=2)
                    elif line_index == 2:
                   
                        ax.plot(line[:, 0], line[:, 1], color='green', linewidth=2)
        
      
        ax.autoscale(tight=True)
        
        ax.set_title(f'Sweep Algorithm Step {scene_index + 1}')
        
        ax.legend(loc='best')
        
        fig.canvas.draw()
        image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
        image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        frames.append(image)
    
    plt.close(fig)
    
    frames_pil = [Image.fromarray(frame) for frame in frames]
    frames_pil[0].save(
        output_filename, 
        save_all=True, 
        append_images=frames_pil[1:], 
        duration=duration, 
        loop=loop
    )
    
    return output_filename

In [145]:
with open('4.json', 'r') as file:  #dziala
    segments = json.loads(file.read())

result, scenes, point = is_intersect(segments)
print(result, str(point))

True [-20.118993111466644, -50.21809769277371]


In [26]:
# tworzenie gifow  do klikania / automatycznych
plot2 = Plot(scenes=scenes)
plot2.draw()
#create_sweep_gif(plot2, '1_przeciecie_C.gif')

#### W jaki sposób zaimplementowałeś struktura stanu (stan miotły) oraz struktura zdarzeń w Twoim programie?

In [ ]:
# zdarzenia - priorityqueue Q = (x,side,segment,point,*) 
# Q - x - współrzędna zdarzenia, side - 0 początkek, 1- koniec, 2 - przecięcie, 
# segment - odcinek, point - obiekt Point reprezentujący odcniek
# 5 pole dla następnej funckji

# mitoła - SortedSet T przechowuje odcinki przez które przechodzi miotłą
# Dodajemy T.add(line) i się ustawia albo T.remove()
# przecięcia sąsiednie odcinki sa analizowane

#Line.set(x) przestawia miotłe
#W dowolnym momencie, jeśli dwa odcinki mogą się przecinać, muszą być bezpośrednio sąsiednie w zbiorze T.
#Wynika to z faktu, że:
#Jeśli odcinek A znajduje się pomiędzy odcinkami B i C, to przecięcie między B i C jest niemożliwe,
#dopóki nie usuniemy A (bo A fizycznie uniemożliwia kontakt między nimi).

### Dla wielu przecięc

In [128]:
def intersection(l, k, used, result, cut, Q):
    if (l, k) in used or (k,l) in used: return
    result.append((cut.x, cut.y))
    used.add((l, k))
    Q.put((cut.x, 2, l, cut, k)) # typ 2 - przecięcie

In [127]:
def valid_cut(idx1, idx2, T, used, result, Q, x1):
    cut = intersect(T[idx1], T[idx2])
    if cut is not None and cut.x > x1: # czy przeciecie za miotła
        intersection(T[idx1], T[idx2], used, result, cut, Q)

Algorytm znajdujący wszystkie przecięcia

In [150]:
from sortedcontainers import SortedSet
# zdarzenia - priorityqueue Q = (x,side,segment,point,l) 
# Q - x - współrzędna zdarzenia, side - 0 początkek, 1- kkoniec, 2 - 
# segment - odcniek, point - obkiet Point reprezentujący odcniek
# l - drugi odcinek z ktorym sie przecina

#aktualna wersja algorytmu zostałą zmieniona na pobierającą dane z JSONÓW żeby łatwiej było analizować wielokąty w tych plikach

def all_intersections(data):    
    segments = load(data)
    Q = PriorityQueue() # zdarzenia
    T = SortedSet() # analizowane odcinki
    used = set() # już użyte
    result = []
    size = 0
    lin_col = Lines_Coll(data)
    scenes = [Scene(lines=[lin_col])]
    for segment in segments: size = classify_to_pq(segment, Q, size)
    Line.set_x(0)
    while Q.qsize()>0:
        x1, side, _, p, k = Q.get()
        line = segments[p.i1] # odcinek
        broom = [(p.x, -size), (p.x, size)]
        add_scene(p, result, data, broom, scenes, T,2)
        if side == 0: # początek odcinka
            Line.set_x(x1)
            T.add(line)
            added = T.index(line)
            add_scene(p, result, data, broom, scenes, T,2)
            if added > 0: valid_cut(added, added-1, T, used, result, Q, x1)
            if added < len(T) - 1: valid_cut(added, added+1, T, used, result, Q, x1)
        if side == 1: # koniec odcinka
            idx = T.index(line)
            Line.set_x(x1)
            if idx > 0 and idx < len(T) - 1: valid_cut(idx-1, idx+1, T, used, result, Q, x1)       
            T.remove(line)
            add_scene(p, result, data, broom, scenes, T,2)
        if side == 2: # mamy przecięcie
            T.remove(line)
            T.remove(k)
            Line.set_x(x1+1E-9)
            T.add(line)
            T.add(k)
            idx1, idx2 = T.index(line), T.index(k)
            #if idx2 < idx1: idx1, idx2 = idx2, idx1   
            idx1,idx2 = min(idx1,idx2), max(idx2,idx1)
            if idx1 > 0: valid_cut(idx1, idx1-1, T, used, result, Q, x1)   
            if idx2 < len(T) - 1: valid_cut(idx2+1, idx2, T, used, result, Q, x1)       
            add_scene(p, result, data, broom, scenes, T,2)
    #print(result)
    return result, scenes


In [151]:
with open('7.json', 'r') as file:
    segments = json.loads(file.read())
#print(segments)
result, scenes = all_intersections(segments)
print((len(result)))

9


In [ ]:
plot3 = Plot(scenes=scenes)
#plot3.draw()
#create_sweep_gif(plot3, '7.gif')